<a href="https://colab.research.google.com/github/hannesMue/ABDA2019/blob/master/Neo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Import der Testdaten von GitHub**

In [52]:
#!rm -rf SparkTestsData/
#!ls SparkTestData

ls: cannot access 'SparkTestData': No such file or directory


In [53]:
!git clone https://github.com/hannesMue/SparkTestsData.git

Cloning into 'SparkTestsData'...
remote: Enumerating objects: 70, done.
remote: Counting objects: 100% (70/70), done.
remote: Compressing objects: 100% (64/64), done.
remote: Total 70 (delta 19), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (70/70), done.


In [37]:
!ls SparkTestsData/Testdaten

eth_eur.CSV  ltc_usd.CSV  test


**Installation Spark**


In [38]:
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install findspark

In [39]:
!ls spark-2.4.4-bin-hadoop2.7
!apt-get install tree

bin   data	jars	    LICENSE   NOTICE  R		 RELEASE  yarn
conf  examples	kubernetes  licenses  python  README.md  sbin
Reading package lists... Done
Building dependency tree       
Reading state information... Done
tree is already the newest version (1.7.0-5).
The following package was automatically installed and is no longer required:
  libnvidia-common-430
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 32 not upgraded.


**Installation von was!?**


In [40]:
!pip install ts ts-flint

In [0]:
import findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"
findspark.init()
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.sql import SQLContext

#spark = SparkSession.builder.config('spark.driver.extraClassPath', '/path/to/postgresql.jar').getOrCreate()

spark = SparkSession.builder.master("local[*]").getOrCreate()

sc = spark.sparkContext
sqlContext = SQLContext(sc)

**Testen von Spark Umgebung**

In [42]:
df = spark.createDataFrame([{"hello": "world"} for x in range(1000)])
df.show(3)
df.count()

/content/spark-2.4.4-bin-hadoop2.7/python/pyspark/sql/session.py:346: UserWarning: inferring schema from dict is deprecated,please use pyspark.sql.Row instead
  warnings.warn("inferring schema from dict is deprecated,"


+-----+
|hello|
+-----+
|world|
|world|
|world|
+-----+
only showing top 3 rows



1000

**Überprüfen der Daten**

In [54]:
df_spark_eth = spark.read.csv('./SparkTestsData/Testdaten/eth_eur.CSV', inferSchema=True, header=True)
from pyspark.sql.functions import input_file_name
df_spark_eth = df_spark_eth.withColumn("filename",input_file_name())

print(type(df_spark_eth))
df_spark_eth.printSchema()
df_spark_eth.show()
df_spark_eth.count()

!head -n 2 ./SparkTestsData/Testdaten/eth_eur.csv

<class 'pyspark.sql.dataframe.DataFrame'>
root
 |-- Datum: string (nullable = true)
 |-- Timestamp: integer (nullable = true)
 |-- Closing: integer (nullable = true)
 |-- filename: string (nullable = false)

+----------+----------+-------+--------------------+
|     Datum| Timestamp|Closing|            filename|
+----------+----------+-------+--------------------+
|01.11.2019|1572566400|    165|file:///content/S...|
|02.11.2019|1572652800|    164|file:///content/S...|
|03.11.2019|1572739200|    165|file:///content/S...|
|04.11.2019|1572825600|    162|file:///content/S...|
|05.11.2019|1572912000|    168|file:///content/S...|
|06.11.2019|1572998400|    175|file:///content/S...|
|07.11.2019|1573084800|    170|file:///content/S...|
|08.11.2019|1573171200|    169|file:///content/S...|
|09.11.2019|1573257600|    167|file:///content/S...|
|10.11.2019|1573344000|    171|file:///content/S...|
|11.11.2019|1573430400|    171|file:///content/S...|
|12.11.2019|1573516800|    168|file:///content/S..

In [67]:
df_spark_eth.createOrReplaceTempView("eth")
#Einige Versuche:

#df_spark_eth.select(to_date(eth.Datum).alias('Datum')).collect()

#basis_daten = spark.sql(
#    "SELECT Datum,CAST(Datum As Date), Closing, AS formatted_date FROM eth")

#Lösung: CSV noch zusätzlich mit Timestamp aufgefüllt

#Spalte für Closing vom Vortag erzeugen um 
basis_daten = spark.sql(
    "SELECT Datum,Timestamp,CAST(Timestamp AS Timestamp) AS formatted_date, Closing FROM eth" 
    #WHERE a.formatted_date ORDER BY formatted_date DESC "
)

basis_daten.show()

+----------+----------+-------------------+-------+
|     Datum| Timestamp|     formatted_date|Closing|
+----------+----------+-------------------+-------+
|01.11.2019|1572566400|2019-11-01 00:00:00|    165|
|02.11.2019|1572652800|2019-11-02 00:00:00|    164|
|03.11.2019|1572739200|2019-11-03 00:00:00|    165|
|04.11.2019|1572825600|2019-11-04 00:00:00|    162|
|05.11.2019|1572912000|2019-11-05 00:00:00|    168|
|06.11.2019|1572998400|2019-11-06 00:00:00|    175|
|07.11.2019|1573084800|2019-11-07 00:00:00|    170|
|08.11.2019|1573171200|2019-11-08 00:00:00|    169|
|09.11.2019|1573257600|2019-11-09 00:00:00|    167|
|10.11.2019|1573344000|2019-11-10 00:00:00|    171|
|11.11.2019|1573430400|2019-11-11 00:00:00|    171|
|12.11.2019|1573516800|2019-11-12 00:00:00|    168|
|13.11.2019|1573603200|2019-11-13 00:00:00|    169|
|14.11.2019|1573689600|2019-11-14 00:00:00|    169|
|15.11.2019|1573776000|2019-11-15 00:00:00|    165|
|16.11.2019|1573862400|2019-11-16 00:00:00|    164|
|17.11.2019|